#### Zillow Regression Analysis
**Artifact: Jupyter Notebook Report**

Created by: Mijail Q. Mariano

Wednesday, July 27th 2022

-----

In [ ]:
# notebook dependencies
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

# key libraries
import pandas as pd
import numpy as np
import scipy.stats as stats
from math import sqrt

# visualization libraries
import matplotlib as mlp
mlp.rcParams['figure.dpi'] = 300
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

# created zillow .py files and functions
from prepare import display_all
import acquire
from acquire import get_zillow_dataset, \
            clean_zillow_dataset, \
            zillow_outliers, \
            plot_target, \
            plot_continuous, \
            plot_discrete, \
            train_validate_test_split, \
            select_kbest, \
            recursive_feature_eng


# sklearn library for modeling
from sklearn.metrics import mean_squared_error
from sklearn.feature_selection import SelectKBest, RFE, RFECV
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression, LassoLars, TweedieRegressor
from sklearn.preprocessing import RobustScaler, StandardScaler, MinMaxScaler

----

### <u>**``Focal Analysis Questions:``**</u>

**``Target Variable:``**
* Tax_Value_Dollar_Count = **"home_value"**


**1. Does a home's location matter?**

I presume location matters in determining the final value/price of a home, as there could be economic factors (e.g., cost-of-living/inflationary considerations) that may be considered when assessing a home's relative value. Additionally, homes typically closer to metropolitan or labor producing areas may be more costly than rural/less-labor promoted areas.

* Homes closer to schools (predicted positive relationship)
* Home closer to parks/recreational areas (predicted positive relationship)
* Homes near hospitals/hospice care (predicted positive relationship)
* Areas with relatively less crime rates or high law-enforcement presence. (predicted negative relationship)
* Given the additional insurance costs/natural disaster considerations homes near bodies of water may be less attractive to home buyers. (predicted negative relationship)

``Relative Features:``
1. Fips
2. Latitude 
3. Longitude 
4. Parcel_id
5. Region_City_id
6. Region_id_County
7. Region_id_Zip


**2. Does home space matter?**

I presume that the larger the home (as measured by sq. ft.), the higher the home value will be. 

A larger home can be more attractive to home buyers since the space can act as both an initial family home, but also their “forever home”. Meaning that people who might not initially have a use for the additional space - may see the potential benefits of having it when they are ready to either 1. expand their family/or find use for the space or 2. View the space as a future investment for a buyer willing to pay the same or more for their spacious home in the future (investment thinking). 

* Consider the total number of baths 
* Consider the total number of bedrooms

``Relative Features:``
1. Calculated_Finished_Sq_Feet
2. Bathroom_Count
3. Bedroom_Count
4. Full_Bath_Count


**3. Does the home selling or purchase period matter?**

I presume that the period when a home is purchased or placed on the market will matter. Home buyers may be more reluctant to purchase a home in the colder regional months (e.g., typically winter) when the weather may be less favorable for moving. 

There may also be *renter factors or periods in the year when leases end and renters make the decision to purchase a home, subsequently driving more buyers to the market and thus potentially increasing home sale values. 

*(higher demand + “same” or not enough supply = more competition/higher home purchase price)*

* Consider seasonal patterns (e.g., summer months vs. winter) 

``Relative Features:``
1. Transaction_Date
2. Year_built
3. Assessment_Year


**<u>Features not taken forward after initial query pull:</u>**

- **parcel_id** 
  - Though may be needed in final predictions/report
<br> </br>

- **room_count** 
  - This is the total number of rooms in a home. Omitting this since it may be too closely associated to other features such as bathrooms and bedrooms. Additionally, this analysis presumes that all homes have standard rooms such as kitchens and living-rooms.
<br> </br>
- calculated bath and bedroom
- full bath count
- census tract and block
- finished squared feet 12
- property county land use code
- raw census tract and block
- assessment tax year
- land tax value dollar count
- structure tax value dollar count
- tax amount
- structure tax value dollar count
- assessment tax year

----
### **``Data Acquisition and Preparation``**

**``Key Highlights``**

- Used domain knowledge and research to focus on key questions for analysis
- Renamed and converted columns/features to proper type
- Dropped initial features with > 20% Null values
- Labeled and omitted homes with recorded finished sq. footage > 8K as larger/outlier homes*
- Focused on home values within the 90th percentile ~900K

*these homes represented homes larger than the majority homes in the dataset, therefore making the analysis or future prediction less accurate or obscured when including these records in the analysis.* *


In [ ]:
# acquiring and preparing initial zillow dataset

df = get_zillow_dataset()
df = clean_zillow_dataset(df)
df = zillow_outliers(df)
df.shape

In [ ]:
# confirming the dataset
df.head()

In [ ]:
# confirming the data types
pd.DataFrame(df.dtypes.sort_values()).rename(columns = {0: "data_type"})

In [ ]:
# summary statistics for continuous variables/features
summary_stats = df.describe().T
summary_stats["range"] = summary_stats["max"] - summary_stats["min"]
summary_stats.round()

----
**``Initial Univariate Feature Analysis:``**

In [ ]:
# exploring/visualizing the target - tax dollar home value ("home_value")

plt.figure(figsize = (14, 8))
sns.set(font_scale = 1)
ax = sns.histplot(df["home_value"], kde = True)

# removing axes scientific notation 
ax.ticklabel_format(style = "plain")

plt.axvline(df["home_value"].mean(), linewidth = 2, color = 'purple', alpha = 0.5, label = "mean")
plt.text(335500, 1400, "$333,952", horizontalalignment='left', size='medium', color='purple', weight='semibold')
plt.ylabel("Frequency")
plt.xlabel("Home Value")
plt.legend()
plt.title("Home Value Distribution")
plt.show()

In [ ]:
# home values box plot
plt.figure(figsize = (10, 6))
plt.boxplot(df["home_value"])

ax.set(xlabel = None)
plt.title("Home Values: Remaining Outliers")
plt.xlabel(None)
plt.show()

In [ ]:
# total makeup of homes by value bin
plot_target(df["home_value"])

In [ ]:
# plotting continuous/numerical features
plot_continuous(df)

In [ ]:
# plotting discreate/categorical features
plot_discrete(df)

<u>``Note on 'county_id':``</u>

* the sns.countplot for 'county_id' shows a 1for1 frequency match with fips_code
* omitting county_id as this is may be too closely associated with fips_code and is currently more ambigous in exact location

In [ ]:
# creating a new feature/variable for month transaction/purchases in 2017
df = acquire.get_months_and_plot(df)

----
### <u>``Pre-processing the Data:``</u>

**``Highlights``**

- Established individual feature class (continuous/discrete)
- Generated new features potentially more descriptive of the data
- Created dummy columns/variables for discrete features
- Dropped redundant columns/features

In [ ]:
unique_lst = []
for col in df.columns:
    if df[col].nunique() > 20:
        output = {
            'Feature': col,
            'Unique Values': df[col].nunique(),
            'Feature Class': "continuous"}
    else:
        output = {
            'Feature': col,
            'Unique Values': df[col].nunique(),
            'Feature Class': "discrete"}
    
    unique_lst.append(output)

pd.DataFrame(unique_lst).sort_values('Unique Values', ascending = False).reset_index(drop=True)

In [ ]:
# creating a new column to calculate total number of elapsed years through current year
df = acquire.age_of_homes(df)

In [ ]:
# creating column to clean up month-year column
df = acquire.clean_months(df)

In [ ]:
# creating new columns for binning bathrooms and bedrooms
df = acquire.bin_bath_and_beds(df)
df.shape
# df.columns.tolist()

In [ ]:
# creating new column for half baths
df = acquire.get_half_baths(df)
print(f'dataframe shape: {df.shape}')
df.columns.tolist()

In [ ]:
# Creating Dummy Columns/Variables for Discrete/Categorical Features
dummy_df = pd.get_dummies(data = df, columns = ['fips_code', 'purchase_month'])
df = dummy_df.copy()

In [ ]:
# dropping redundant columns & 'county_id' column
df = acquire.drop_after_dummy(df)
print(f'New Dataframe W/Dummy Variables: {df.shape}')
df.columns.tolist()

-----
#### **Splitting Zillow Dataset for Exploration/Hypothesis Testing:**

In [ ]:
train, validate, test = train_validate_test_split(df)

In [ ]:
# establishing a home value baseline prediction
train, validate = acquire.establish_baseline(train, validate)

In [ ]:
# dataset subsplits (x and y variables)
X_train = train.drop(columns = "home_value")
y_train = train['home_value']

X_validate = validate.drop(columns = "home_value")
y_validate = validate['home_value']

X_test = test.drop(columns = "home_value")
y_test = test['home_value']

----
### **``Exploration and Hypothesis Testing``**

**<u>``Two Sample T-test Hypothesis:``</u>**
* $H_0$ Null Hypothesis: The home value average for all feature values are the same
* $H_a$ Alternative Hypothesis: The home value average across feature values are different

**<u>``Pearson R Correlation Hypothesis:``</u>**
* $H_0$ Null Hypothesis: There is no linear correlation between the variables
* $H_a$ Alternative Hypothesis:  There is a linear correlation between variables

$alpha$: 0.05

<u>**Definitions:**</u>

The *"t_score"* is a ratio between the difference of two groups and the difference within the groups:
- Larger t_score = more difference between groups
- Smaller t_score = more similarity between groups

The *"p_value"* is the evidence against a null hypothesis: 

- Smaller p_value = the stronger the evidence that you should reject the null hypothesis


|feature|class|test|
|----|----|----|
|'bathroom-binned'|discrete|ind. t-test|
|'bedroom-binned'|discrete|ind. t-test|
|'fips_code'|discrete|ind. t-test|
|'purchase_month|discrete|ind. t-test|
|'living_sq_feet'|continuous|pearson-r|
|'latitude'|continuous|pearson-r|
|'longitude'|continuous|pearson-r|
|'property_sq_feet'|continuous|pearson-r|
|'city_id'|continuous|pearson-r|
|'zip_code'|continuous|pearson-r|
|'year_built'|continuous|pearson-r|
|'home_age'|continuous|pearson-r|


----

In [ ]:
# plotting continuous variables against target
cont_lst = train.select_dtypes(exclude = ["bool", "uint8", "object"]).columns.tolist()
cont_df = train[cont_lst]
acquire.plot_variable_pairs1(cont_df)

In [ ]:
# plotting non-dummy variables against target
pair_lst = train.select_dtypes(exclude = ["bool", "uint8"]).columns.tolist()
acquire.plot_variable_pairs2(train, pair_lst)

In [ ]:
cat_lst = train.select_dtypes(include = ["bool", "uint8"]).columns.tolist()
cont_lst = train.select_dtypes(exclude = ["object", "uint8", "bool"]).columns.tolist()

target_mean = round(y_train.mean(), 2)
alpha = 0.05

metrics = []
for col in cat_lst:
    sub_group = train[train[col] == 1].home_value
    t_score, p_value = stats.ttest_1samp(sub_group, target_mean)

    if p_value < alpha:
        output = {
            "discrete feature": col,
            "t_score": t_score,
            "p_value": p_value}
        
        metrics.append(output)

    else:
        print(f'Column: {col} not statistically significant.')
        print("---------------------------------------------------------------")

categorical_scores = pd.DataFrame(metrics)
categorical_scores.round(4)

In [ ]:
target = train["home_value"]
alpha = 0.05

metrics = []

for col in cont_lst:
    r, p_value = stats.stats.pearsonr(train[col], target)

    if p_value < alpha:
        output = {
            "continuous feature": col,
            "correlation coeffficient": r,
            "p_value": p_value}
        
        metrics.append(output)

    else:
        print(f'Column: {col}')
        print(f'P_value: {round(p_value, 4)}')
        print('Not statistically significant.')
        print("---------------------------------------------------------------")

continuous_scores = pd.DataFrame(metrics)
continuous_scores.round(4)

----
**``Statiscal Analysis/Conclusion Notes:``**

* Given the low/no statistical significance found in the following features or values, I elect to drop or disregard the following feature options in the datasets used in modeling. 
* Looking ahead, I will want to further examing/explore why these feature or options are considered "statistically insignificant" when determining "home_value".

<u>``Continuous features:``</u>
1. "city_id"
2. "property_sq_feet"
3. "longitude"
4. "zip_code"

<u>``Categorical/discrete values:``</u>
1. '4_to_6.5_baths'
2. '3_to_4_bedrooms'
3. '5_to_6_bedrooms'
4. 'purchase_month_February'
5. 'purchase_month_June'
6. 'county_id'*

county_id

In [ ]:
print(f'X_train shape: {X_train.shape}')
print(f'X_validate shape: {X_validate.shape}')
print(f'X_test shape: {X_test.shape}')

In [ ]:
# cleaning independent feature datasets -- preparing for modeling:
X_train = acquire.clean_for_features(X_train)
X_validate = acquire.clean_for_features(X_validate)
X_test = acquire.clean_for_features(X_test)

print(f'X_train shape: {X_train.shape}')
print(f'X_validate shape: {X_validate.shape}')
print(f'X_test shape: {X_test.shape}')

----
### **``Modeling``**

In [ ]:
# distribution of continuous variables scaled: using SKlearn Robust Scaler
cont_lst = X_train.select_dtypes(exclude = ["object", "uint8", "bool"]).columns.tolist()

for col in cont_lst:
    scaler = RobustScaler()
    scaler.fit(X_train[[col]])

    x_scaled = scaler.transform(X_train[[col]])

    plt.figure(figsize=(18, 6))
    plt.subplot(121)
    ax = sns.histplot(X_train[[col]], bins = 25, edgecolor = 'black', label = col)
    # removing axes scientific notation 
    ax.ticklabel_format(style = "plain") 
    plt.title(f'Original: {col}')
    plt.legend()

    plt.subplot(122)
    ax = sns.histplot(x_scaled, bins=25, edgecolor = 'black', label = "scaled")
    # removing axes scientific notation 
    ax.ticklabel_format(style = "plain") 
    plt.title(f'Scaled: {col}')
    plt.legend()

In [ ]:
# scaling necessary features
X_train = acquire.scaled_data(X_train)
X_validate = acquire.scaled_data(X_validate)
X_test = acquire.scaled_data(X_test)

In [ ]:
# confirming the scaling
X_train.home_age.describe() # checks out!

In [ ]:
# Running K_best function for top 10 independent features in training dataset
pd.DataFrame(acquire.select_kbest(X_train, y_train, 10)).rename(columns = {0: "Feature"})

In [ ]:
# Running Sklearn Recursive Function for top 10 grouped functions in dataset
acquire.recursive_feature_eng(X_train, y_train, 10).reset_index(drop=True)

**``A Note on Sklearn's Feature Engineering "RFECV" Function``**

<u>**Cross-validation Estimator:**</u>

* An estimator that has built-in cross-validation capabilities to automatically select the best hyper-parameters. 
* Advantages of using a cross-validation estimator is that they can take advantage of warm-starting by reusing precomputed results in the previous steps of the cross-validation process. This generally leads to speed improvements. 
* By default, all these estimators, apart from RidgeCV with an LOO-CV, will be refitted on the full training dataset after finding the best combination of hyper-parameters.

<u>**Scorer:**</u>

* A non-estimator callable object which evaluates an estimator on given test data, returning a number. Unlike evaluation metrics, a greater returned number must correspond with a better score. See The scoring parameter: defining model evaluation rules.

In [ ]:
# initiating and fitting Sklearn's RFECV feature selection function
RFECV = RFECV(
    estimator = LinearRegression(),
    min_features_to_select = 5)

RFECV = RFECV.fit(X_train, y_train)

In [ ]:
# selected number of features
feature_lst = X_train.columns[RFECV.support_].tolist()
pd.DataFrame(feature_lst).rename(columns = {0: "Features"}).sort_values("Features").reset_index(drop=True)

**``Generating Models & Scoring against Train Dataset: Linear Regression, Lasso Lars, and Tweedie Regressor``**

In [ ]:
lr = LinearRegression()
lr_model = lr.fit(X_train, y_train)

lars = LassoLars()
lars_model = lars.fit(X_train, y_train)

glm = TweedieRegressor(alpha = 1, power = 0)
glm_model = glm.fit(X_train, y_train)

print("Training R-squared w/Linear Regression:", lr_model.score(X_train, y_train).round(4))
print("Training R-squared w/Lasso Lars:", lars_model.score(X_train, y_train).round(4))
print("Training R-squared w/Tweedie Regressor:", glm_model.score(X_train, y_train).round(4))

**``Deploying Models on Validate Dataset: Mean Squared Error (MSE)``**

In [ ]:
models = [lr_model, lars_model, glm_model]

for model in models:

    train_model = model.predict(X_train)
    rmse_train = sqrt(mean_squared_error(y_train,
                                         train_model))
    
    validate_model = model.predict(X_validate)
    rmse_validate = sqrt(mean_squared_error(y_validate,
                                         validate_model))
    
    print('RMSE for {} model on the train dataset: {}'.format(model, round(rmse_train, 2)))
    print('RMSE for {} model on the validate dataset: {}'.format(model, round(rmse_validate, 2)))
    print()

-----
#### <u>**``Model Plots and Evaluation on Validate Dataset:``**</u>

In [ ]:
# creating the independent and dependent variables
X_var = pd.DataFrame(X_validate[feature_lst])
y_var = pd.DataFrame(y_validate)
model_df = pd.concat([X_var, y_var], axis = 1).reset_index(drop = True)

# creating a home value mean baseline prediction
baseline_mean_predictions = round(y_validate.mean(), 2)
model_df["baseline_mean_predictions"] = baseline_mean_predictions

model_df.head()

In [ ]:
# generating validate model predictions and assigning to dataframe
lr_predictions = lr_model.predict(X_validate)
model_df["linear_predictions"] = lr_predictions.round(2)

lars_predictions = lars_model.predict(X_validate)
model_df["lars_predictions"] = lars_predictions.round(2)

glm_predictions = lars_model.predict(X_validate)
model_df["glm_predictions"] = glm_predictions.round(2)

In [ ]:
# generating error reports for all models and baseline:
SSE, ESS, TSS, MSE, RMSE = acquire.get_error_report(model_df["home_value"], model_df["linear_predictions"])
print()
SSE, ESS, TSS, MSE, RMSE = acquire.get_error_report(model_df["home_value"], model_df["lars_predictions"])
print()
SSE, ESS, TSS, MSE, RMSE = acquire.get_error_report(model_df["home_value"], model_df["glm_predictions"])
print()
SSE, ESS, TSS, MSE, RMSE = acquire.get_error_report(model_df["home_value"], model_df["baseline_mean_predictions"])

In [ ]:
# acquiring melted model column for easier plotting
melt_df = acquire.get_melted_table(model_df)

In [ ]:
# plotting model residuals
acquire.plot_model_residuals(melt_df)

In [ ]:
# plotting model predicted home values against target
acquire.plot_models(melt_df)

In [ ]:
# Actual Home Values and Model Predicted Values
acquire.model_distributions(model_df)

### <u>``Deploying Linear Regression Model on Test Dataset:``</u>

In [ ]:
# generating a dataframe
test_df = pd.DataFrame(y_test)

# generating model predictions
test_df["model_predictions"] = lr_model.predict(X_test).round(2)
test_df.head()

In [ ]:
# returning R-squared score & RMSE on test dataset
rmse_test = sqrt(mean_squared_error(test_df['home_value'], test_df['model_predictions']))
print("Training R-squared w/Linear Model:", lr_model.score(X_test, y_test).round(4))
print(f'RMSE for OLS model on the test dataset: {round(rmse_test, 2)}')

In [ ]:
# returning RMSE report 
acquire.final_rmse()

----
**<u>``Analysis Summary``</u>**

Overall, the linear regression model performed at ~16% better accuracy than a baseline mean home value predictor. Though not entirely conclusive of a home's tax assessed value - I believe this model may be able to handle fluctuations in the overall market particularly well. 

By using "binned" or categorical features in traditionally sought after home characteristics (e.g., bedrooms, bathrooms) to determine a home's value, the model helps to handle external factors such as seasonality/seasonal effects, cultural preferences, or demand shocks that can undoubtedly impact the number of 'for sale' homes available and subsquently, house prices in relatively short periods.


**``Recommendations:``**

1. Create a **"Real-estate Training Program** that aims at helping Real-estate Brokers/Agents, Marketing, and Real-estate Consultancy teams to familiarize themselves with seasonal patterns in their local areas. By offering this program to real-estate professionals who are often closest to both sellers and buyers, it would help them to:

 - Better advise their clients on the most optimal periods to enter the market (purchase/sell their home)
 - More quickly recognize housing market shocks and make real-time decisions that help to normalize home value prices 

2. Use our online, mobile application, and advisory platforms to promote renovations of not just older homes, but smaller spaces and potentially converting them to **half-baths**. This feature along with more finished living space appears to be appealing characteristics for home buyers trading in traditional features such as the number of bedrooms or bathrooms for more efficient and univarsal home space.

----

**Looking Ahead (next steps):**

- improve the model's predictive accuracy by identifying, testing, and including other potential home market factors 
    - regional cost-of-living indices
    - unemployment rates
    - educational/school ratings
    - crime rates
    - home design styles
<br>
- calculate home/area distance to nearby metropolitan cities, park/recreational areas, schools, hospitals/hospice centers, etc.
- parse out fips codes into more distinct locations either by towns/villages/neighborhoods or exact cities 